In [19]:
import sys
sys.path.append("../src")

In [5]:
from preprocessing.data_sim import *
from model.model import *
from validation.CV import *

ModuleNotFoundError: No module named 'preprocessing'

In [22]:
model = Model([LogisticRegressionCV(penalty='l2', solver='newton-cg', multi_class='multinomial'),
               LogisticRegressionCV(penalty='l1', solver='liblinear'),
               RandomForestClassifier(),
               RFECV(estimator=SVC(gamma="scale", kernel="linear"), verbose=1),
               RFECV(estimator=LDA(), verbose=1),
              ])
model

Components:

('Component_0', LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='multinomial', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='newton-cg', tol=0.0001, verbose=0))

('Component_1', LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l1', random_state=None, refit=True, scoring=None,
                     solver='liblinear', tol=0.0001, verbose=0))

('Component_2', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.

In [23]:
def data_sim2(no_features=50, no_samples=600, delta_factor=1, starts=[[20, 10, 20], [10, 20, 30]], sizes=[[5, 5, 5], [5, 5, 5]], random_labels=False):
    mean = uniform(low=-1.0, high=1.0, size=(no_features,))
    eigs = uniform(low=1.0, high=10.0, size=(no_features,))
    s = np.diag(eigs)
    q, _ = la.qr(uniform(low=10.0, high=100.0, size=(no_features, no_features)))
    variance = q.T @ s @ q

    takes = multivariate_normal(mean, variance, no_samples)

    X = [takes[:, int(l):int(l + r)] for l, r in zip(starts[0], sizes[0])]
    X2 = [takes[:, int(l):int(l + r)] for l, r in zip(starts[1], sizes[1])]

    alpha = [uniform(low=-1.0, high=1.0, size=(s,)) for s in sizes[0]]
    alpha2 = [uniform(low=-1.0, high=1.0, size=(s,)) for s in sizes[1]]
    delta = random(len(sizes[0])) * delta_factor

    val = [np.dot(x, a) for x, a in zip(X, alpha)]
    val2 = [np.dot(x, a) for x, a in zip(X2, alpha2)]
    
    label = [np.logical_or(v > d, v2 > d) for v, v2, d in zip(val, val2, delta)]

    arr = np.empty(no_samples, int)

    for i in range(no_samples):
        if label[0][i] == True:
            arr[i] = 1
        elif label[1][i] == True:
            arr[i] = 2
        elif label[2][i] == True:
            arr[i] = 3
        else:
            arr[i] = 4

    if random_labels:
        arr = randint(1, 5, no_samples)

    return takes, arr

In [24]:
X, y = data_sim2()

print(y)

X_t = X[300:,:]
y_t = y[300:]
X = X[:300,:]
y = y[:300]

[2 4 1 1 2 1 1 2 1 1 2 1 2 2 1 1 2 1 1 1 2 2 1 2 1 1 1 2 1 1 2 1 1 1 4 2 1
 2 1 1 3 1 1 1 2 1 1 1 1 1 1 2 2 2 1 2 1 1 1 1 1 2 1 2 1 1 1 1 1 2 1 2 1 1
 1 1 2 1 1 2 1 4 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 2 1 2 4 2 1 1 1 1
 1 1 1 1 1 2 1 2 2 1 1 1 2 2 1 2 2 1 2 2 1 1 2 1 1 1 4 1 1 1 1 1 2 4 1 1 2
 2 2 3 1 2 1 1 2 1 2 3 2 1 1 1 1 1 1 1 1 2 2 4 1 1 1 1 2 4 1 1 1 1 1 1 1 1
 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 1 1 1 2 2 1 3 1 1 1 1 3 1 3 2
 1 1 2 1 1 1 1 3 1 1 1 1 1 1 2 2 1 1 1 1 2 2 1 1 2 1 1 2 1 2 2 1 1 2 1 1 1
 2 1 1 1 1 2 1 1 2 1 1 1 1 2 1 1 3 1 1 2 1 1 1 3 1 3 1 1 3 1 2 2 2 1 1 1 1
 1 2 2 1 1 3 1 2 1 2 1 1 2 1 2 1 3 3 1 3 2 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 3 1 2 1 2 1 1 1 2 1 2 2 2 1 2 1 1 1 1 1
 2 1 1 2 1 2 1 3 1 2 1 2 1 2 1 1 2 1 2 1 1 2 1 1 3 1 2 1 1 2 1 1 1 2 2 2 1
 1 2 1 1 1 1 1 1 2 2 1 2 1 1 1 1 1 1 1 2 3 1 1 1 1 1 1 1 1 1 2 2 1 2 1 1 1
 1 1 4 2 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 4 2 1 1 1 1 2 1 1 1 1 3 1 1 1 2 2 2
 1 1 1 2 1 1 1 1 1 1 2 1 

In [25]:
model.fit(X, y)

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning

Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 fe

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features

In [26]:
model.set_validation(Validator())
model.validate(X_t, y_t)

array([{'avg_missclass': 0.18333333333333332, 'avg_acc': 0.8166666666666667, 'avg_prec': 0.8166666666666667, 'avg_F1': 0.8166666666666667, 'avg_scaled_conf_mx': array([[205,  13,   0,   0],
       [ 25,  40,   0,   0],
       [  7,   7,   0,   0],
       [  2,   1,   0,   0]]), 'avg_conf_prob_mx': array([[0.68333333, 0.04333333, 0.        , 0.        ],
       [0.08333333, 0.13333333, 0.        , 0.        ],
       [0.02333333, 0.02333333, 0.        , 0.        ],
       [0.00666667, 0.00333333, 0.        , 0.        ]])},
       {'avg_missclass': 0.18333333333333332, 'avg_acc': 0.8166666666666667, 'avg_prec': 0.8166666666666667, 'avg_F1': 0.8166666666666667, 'avg_scaled_conf_mx': array([[197,  13,   8,   0],
       [ 14,  42,   9,   0],
       [  3,   5,   6,   0],
       [  0,   1,   2,   0]]), 'avg_conf_prob_mx': array([[0.65666667, 0.04333333, 0.02666667, 0.        ],
       [0.04666667, 0.14      , 0.03      , 0.        ],
       [0.01      , 0.01666667, 0.02      , 0.        ],


In [27]:
model.perform_voting()

array([0.20166394, 0.35855131, 0.52811266, 0.10510447, 0.0635375 ,
       0.09646599, 0.30782582, 0.28810271, 0.11669159, 0.11096202,
       0.70032251, 0.85026409, 0.98204964, 0.71812114, 0.32564033,
       0.31459308, 0.28579747, 0.13271315, 0.47668624, 0.32835575,
       0.52375913, 0.87874743, 0.52955149, 0.7972069 , 0.64488944,
       0.0799925 , 0.08639567, 0.31745037, 0.13226182, 0.14807628,
       0.21295417, 0.5098252 , 0.2881239 , 0.25237544, 0.50075558,
       0.18398592, 0.30647409, 0.51469264, 0.12833047, 0.27324201,
       0.15800445, 0.1020903 , 0.10176862, 0.36597984, 0.32011913,
       0.28522906, 0.08987378, 0.39862015, 0.12401117, 0.34777631])

In [28]:
model.feature_ranking()

array([array([[-5.89792671e-03,  6.89818678e-03,  4.84240761e-03,
         2.52469286e-02, -1.11384854e-02,  1.87501941e-04,
         8.35747323e-03,  8.57551553e-03,  7.78900177e-03,
        -6.16578446e-03,  1.19673338e-02, -1.20639241e-01,
        -1.62817242e-01,  1.41307208e-01,  3.31246884e-02,
        -4.05907057e-02,  1.89221641e-02,  3.84469895e-02,
        -2.93027146e-03,  2.62863999e-02,  5.57909961e-02,
        -1.88581465e-01,  8.35715257e-04,  1.08408477e-01,
         7.28729455e-02,  1.29573031e-02,  1.39867236e-02,
         5.10231625e-02, -2.97567444e-02,  1.26022950e-02,
         2.51950751e-02, -3.54504623e-03,  3.22141931e-02,
        -8.26673959e-04,  3.74161285e-02,  2.00580746e-04,
         1.05925113e-02,  1.74394085e-02, -1.82496770e-02,
        -8.65971403e-03, -3.81954323e-02, -2.84554588e-03,
        -2.89843583e-03,  5.02039167e-02,  1.68790755e-02,
         1.09167034e-03,  1.64746800e-02,  3.56565737e-02,
        -3.95608886e-02,  5.38839101e-02],
      

In [40]:
final = model.perform_voting()

for i in range(1, 100):
    model = Model([LogisticRegressionCV(penalty='l2', solver='newton-cg', multi_class='multinomial'),
               LogisticRegressionCV(penalty='l1', solver='liblinear'),
               RandomForestClassifier(),
               RFECV(estimator=SVC(gamma="scale", kernel="linear"), verbose=1),
               RFECV(estimator=LDA(), verbose=1),
              ])
    model.fit(X, y);
    
    final = np.vstack((final,model.perform_voting()))
    

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mateusz/.local/lib/

Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 fe

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 fe

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning

...

Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 fe

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/mateusz/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning

Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 fe

/home/mateusz/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 fe

In [41]:
np.apply_along_axis(np.mean, 0, final)

array([0.18627014, 0.33855728, 0.51726368, 0.1148499 , 0.07865766,
       0.0803053 , 0.320418  , 0.28421886, 0.11142903, 0.13051973,
       0.69586022, 0.84428987, 0.97015572, 0.7645901 , 0.33561553,
       0.31143127, 0.30139031, 0.14262259, 0.48191058, 0.32464972,
       0.49799791, 0.94062   , 0.5364821 , 0.7943982 , 0.65939075,
       0.06835711, 0.1005021 , 0.33111269, 0.13812706, 0.16063529,
       0.21338257, 0.49458261, 0.29880513, 0.26880726, 0.50786831,
       0.16127989, 0.33157912, 0.5082203 , 0.1322408 , 0.28640897,
       0.16620868, 0.1119498 , 0.09997369, 0.38557561, 0.29574581,
       0.29451278, 0.09095061, 0.39184575, 0.140656  , 0.35730309])